In [54]:
%run "./1. Data Loading.ipynb"

X: (800, 360, 25)
Y: (800,)


In [55]:
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from sklearn.model_selection import StratifiedKFold

In [56]:
num_samples = 800
num_features = 360
seq_length = 25
num_classes = 2

num_epochs = 10
num_folds = 5

learning_rate = 0.001
batch_size = 32

In [57]:
class MultiFeatureTimeSeries(nn.Module):
    def __init__(self, n_features, n_classes):
        super(MultiFeatureTimeSeries, self).__init__()
       
        self.conv1 = nn.Conv1d(n_features, 128, 3)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool1d(kernel_size = 2)

        self.fc1 = nn.Linear(1408, 128)
        self.fc2 = nn.Linear(128, n_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [58]:
k_sampler = StratifiedKFold(n_splits=num_folds, shuffle=True)

# Create a SummaryWriter object
writer = SummaryWriter()

for fold_index, (train_indices, test_indices) in enumerate(k_sampler.split(X, Y)):
    print(f"Fold {fold_index + 1}:")

    train_length = len(train_indices)
    test_length = len(test_indices)
    x_train, y_train = X[train_indices], Y[train_indices]
    x_test, y_test = X[test_indices], Y[test_indices]

    model = MultiFeatureTimeSeries(num_features, num_classes)
    model = model.float()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch_index in range(num_epochs):
        print(f"\tEpoch {epoch_index}:")

        train_loss = 0.0
        model.train()

        for batch_index in range(0, train_length, batch_size):
            inputs = torch.from_numpy(x_train[batch_index: batch_index + batch_size]).float()
            labels = torch.from_numpy(y_train[batch_index: batch_index + batch_size]).long()

            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward pass
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        avg_train_loss = train_loss / (train_length // batch_size)
        print("\t\tTrain Loss:", avg_train_loss)

        # Write training loss to TensorBoard
        writer.add_scalar('Loss/Train', avg_train_loss, fold_index * num_epochs + epoch_index)

        model.eval()

        val_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():
            inputs = torch.from_numpy(x_test).float()
            labels = torch.from_numpy(y_test).long()

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            # Compute accuracy
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        # Print validation loss and accuracy
        print(f"\t\tValidation Loss: {val_loss / test_length:.4f}")
        print(f"\t\tValidation Accuracy: {(100 * correct / total):.2f}%")

        # Write validation loss and accuracy to TensorBoard
        writer.add_scalar('Loss/Validation', val_loss / test_length, fold_index * num_epochs + epoch_index)
        writer.add_scalar('Accuracy/Validation', 100 * correct / total, fold_index * num_epochs + epoch_index)

# Close the SummaryWriter
writer.close()


Fold 1:
	Epoch 0:


		Train Loss: 2.5703672289906536
		Validation Loss: 0.0046
		Validation Accuracy: 90.62%
	Epoch 1:
		Train Loss: 0.15689345038308602
		Validation Loss: 0.0011
		Validation Accuracy: 95.62%
	Epoch 2:
		Train Loss: 0.06541210891150442
		Validation Loss: 0.0017
		Validation Accuracy: 95.00%
	Epoch 3:
		Train Loss: 0.043434751016037866
		Validation Loss: 0.0007
		Validation Accuracy: 98.75%
	Epoch 4:
		Train Loss: 0.001955473683810283
		Validation Loss: 0.0005
		Validation Accuracy: 98.75%
	Epoch 5:
		Train Loss: 6.877999044831994e-05
		Validation Loss: 0.0004
		Validation Accuracy: 98.75%
	Epoch 6:
		Train Loss: 4.736031819838615e-05
		Validation Loss: 0.0004
		Validation Accuracy: 98.75%
	Epoch 7:
		Train Loss: 3.982902202430694e-05
		Validation Loss: 0.0004
		Validation Accuracy: 98.75%
	Epoch 8:
		Train Loss: 3.494562061874262e-05
		Validation Loss: 0.0004
		Validation Accuracy: 98.75%
	Epoch 9:
		Train Loss: 3.120432500338666e-05
		Validation Loss: 0.0004
		Validation Accuracy: 98.75%